In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 500)

import numpy as np

import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')

import matplotlib.ticker as ticker

import seaborn as sns
sns.set_style('whitegrid')

from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf


from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from pandas.plotting import autocorrelation_plot

from sklearn import metrics

from sklearn.linear_model import LinearRegression

import warnings

import datetime
from datetime import timedelta, time
import statistics

import matplotlib.dates as mdates
from scipy.stats import pearsonr
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from pylab import rcParams
from matplotlib import rc
%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.5)
rcParams['figure.figsize'] = 16, 10

import matplotlib as mpl
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator  #must use this one for training with TimeSeriesGenerator
%matplotlib inline

In [ ]:
data = pd.read_csv("/kaggle/input/consolidado/consolidado.csv", sep=";")

In [ ]:
#Se filtran los clientes que tienen información completa del 99% en adelante
clientes = ["DATOSCLIENTE1.csv","DATOSCLIENTE2.csv","DATOSCLIENTE3.csv","DATOSCLIENTE4.csv",
            "DATOSCLIENTE5.csv","DATOSCLIENTE6.csv","DATOSCLIENTE7.csv","DATOSCLIENTE8.csv","DATOSCLIENTE9.csv",
            "DATOSCLIENTE10.csv","DATOSCLIENTE16.csv","DATOSCLIENTE17.csv","DATOSCLIENTE18.csv",
            "DATOSCLIENTE19.csv","DATOSCLIENTE20.csv"]

data = data[data['Archivo'].isin(clientes)]

In [ ]:
data_clean = data.copy()
data_clean['Fecha_hora'] = pd.to_datetime(data_clean['Fecha'])
data_clean['Energia_activa'] = data_clean.Active_energy
data_clean['Energia_reactiva'] = data_clean.Reactive_energy
data_clean.drop('Active_energy', axis=1, inplace=True)
data_clean.drop('Reactive_energy', axis=1, inplace=True)
data_clean['Fecha'] = data_clean['Fecha_hora'].dt.date
data_clean['Hora'] = data_clean['Fecha_hora'].dt.time
dayname = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sabado', 'Domingo']
data_clean['Dia_semana'] = pd.to_datetime(data_clean['Fecha']).dt.dayofweek.apply(lambda x: dayname[x])

data_clean.info()

In [ ]:
# Cleaning duplicates for each client
def clean_duplicates_by_client(df):
    # Group by 'Archivo' (cliente) and 'Datetime', and count occurrences
    datetime_counts = df.groupby(['Archivo', 'Fecha_hora']).size()

    # Filter to find where counts are greater than 1
    duplicates = datetime_counts[datetime_counts > 1]

    # Print the result for each client
    for client, counts in duplicates.groupby(level=0):
        print(f"Duplicados de {client}:")
        print(counts)
    # Check for clients without duplicates
    unique_clients = df['Archivo'].unique()
    for client in unique_clients:
        if client not in duplicates.index:
            print(f"No se encontraron duplicados para el {client}")

# Apply the function to the entire dataframe
clean_duplicates_by_client(data_clean)

In [ ]:
# Contar el tamaño de cada grupo por fecha para cada cliente
def count_by_date_for_each_client(df):
    # Iterar sobre cada cliente
    for client, group in df.groupby('Archivo'):
        # Imprimir el tamaño de cada grupo por fecha para el cliente actual
        print(f"Tamaño de cada grupo por fecha para el cliente {client}:")
        print(group.groupby("Fecha").size())

# Aplicar la función al dataframe completo
count_by_date_for_each_client(data_clean)

In [ ]:
# Función para contar el tamaño de cada grupo por hora para cada cliente
def count_by_time_for_each_client(df):
    # Iterar sobre cada cliente
    for client, group in df.groupby('Archivo'):
        # Imprimir el tamaño de cada grupo por hora para el cliente actual
        print(f"Tamaño de cada grupo por hora para el cliente {client}:")
        print(group.groupby("Hora").size())

# Aplicar la función al dataframe completo
count_by_time_for_each_client(data_clean)

In [ ]:
# Función para eliminar las fechas con menos de 24 registros por cliente
def remove_dates_below_24(df):
    # Lista para almacenar los índices de las filas a eliminar
    rows_to_drop = []
    
    # Iterar sobre cada cliente
    for client, group in df.groupby('Archivo'):
        # Group by 'Fecha' and count the records for each date
        grouped = group.groupby("Fecha").size()
        
        # Filtrar las fechas con menos de 24 registros
        dates_below_24 = grouped[grouped < 24]
        
        # Obtener los índices de las filas con esas fechas
        for date in dates_below_24.index:
            rows_to_drop.extend(group[group['Fecha'] == date].index)
    
    # Eliminar las filas con fechas que tienen menos de 24 registros
    df_cleaned = df.drop(rows_to_drop)
    
    return df_cleaned

# Aplicar la función al dataframe completo
data_clean_filtered = remove_dates_below_24(data_clean)

In [ ]:
data_clean_filtered.groupby('Hora').size()

In [ ]:
# Función para convertir la columna 'Energia_activa' a tipo numérico para cada cliente
def convert_to_numeric_for_each_client(df):
    # Iterar sobre cada cliente
    for client, group in df.groupby('Archivo'):
        # Convertir la columna 'Energia_activa' a tipo numérico
        df.loc[group.index, 'Energia_activa'] = pd.to_numeric(group['Energia_activa'], errors='coerce')

# Aplicar la función al dataframe completo
convert_to_numeric_for_each_client(data_clean_filtered)

In [ ]:
# Función para calcular el promedio de 'Energia_activa' para cada hora en cada cliente
def calculate_hourly_mean_for_each_client(df):
    # Lista para almacenar los DataFrames de promedio por hora para cada cliente
    mean_dfs = []
    
    # Iterar sobre cada cliente
    for client, group in df.groupby('Archivo'):
        # Calcular el promedio de 'Energia_activa' para cada hora
        mean_hourly = group.groupby('Hora')['Energia_activa'].mean().reset_index()
        mean_hourly.columns = ['Hora', 'Average_Energia_activa']
        
        # Agregar el DataFrame de promedio por hora del cliente actual a la lista
        mean_dfs.append(mean_hourly)
    
    return mean_dfs

# Obtener los DataFrames de promedio por hora para cada cliente
mean_dfs_per_client = calculate_hourly_mean_for_each_client(data_clean_filtered)

# Imprimir los DataFrames de promedio por hora para cada cliente
for i, df_per_client in enumerate(mean_dfs_per_client):
    print(f"Promedio por hora para el cliente {i+1}:")
    print(df_per_client)
    print()

In [ ]:
# Lista para almacenar los DataFrames individuales de cada cliente
dfs_por_cliente = []

# Iterar sobre cada cliente
for cliente, df_cliente in data_clean_filtered.groupby('Archivo'):
    # Crear un DataFrame para el cliente actual con las columnas 'Fecha_hora' y 'Energia_activa'
    df_cliente_individual = df_cliente[['Fecha_hora', 'Energia_activa']]
    # Establecer 'Fecha_hora' como el índice del DataFrame
    df_cliente_individual = df_cliente_individual.set_index('Fecha_hora')
    # Agregar el DataFrame del cliente actual a la lista
    dfs_por_cliente.append((cliente, df_cliente_individual))

# Imprimir los primeros registros de cada DataFrame
for cliente, df_cliente in dfs_por_cliente:
    print(f"DataFrame para el cliente: {cliente}")
    print(df_cliente.head())
    print()

In [ ]:
##grupo1: 1,2,3,4,5; Grupo2: 6,7,8,10; Grupo3: 9,16,17,18,19,20 
df_cliente1_idx = dfs_por_cliente[0]
df_cliente6_idx = dfs_por_cliente[11]
df_cliente17_idx = dfs_por_cliente[3]

df_cliente1_idx = df_cliente1_idx[1]
df_cliente6_idx = df_cliente6_idx[1]
df_cliente17_idx = df_cliente17_idx[1]

df_cliente1 = df_cliente1_idx.reset_index()
df_cliente6 = df_cliente6_idx.reset_index()
df_cliente17 = df_cliente17_idx.reset_index()

In [ ]:
df_cliente1_idx.info()

In [ ]:
# Check for NaN values in the original DataFrames
print("NaNs in original data:")
print("df_cliente1:", df_cliente1_idx.isna().any())
print("df_cliente6:", df_cliente6_idx.isna().any())
print("df_cliente17:", df_cliente17_idx.isna().any())

In [ ]:
# Scaling for df_cliente1
scaler1 = MinMaxScaler()
scaler_data1 = scaler1.fit_transform(df_cliente1_idx)
# Check for NaNs after scaling
print("NaNs after scaling for df_cliente1:", np.isnan(scaler_data1).any())

In [ ]:
# Scaling for df_cliente6
scaler6 = MinMaxScaler()
scaler_data6 = scaler6.fit_transform(df_cliente6_idx)
# Check for NaNs after scaling
print("NaNs after scaling for df_cliente6:", np.isnan(scaler_data6).any())

In [ ]:
# Scaling for df_cliente17
scaler17 = MinMaxScaler()
scaler_data17 = scaler17.fit_transform(df_cliente17_idx)
# Check for NaNs after scaling
print("NaNs after scaling for df_cliente17:", np.isnan(scaler_data17).any())

In [ ]:
# Create DataFrames
index1 = np.arange(0, len(scaler_data1), 1)
scaler_df1 = pd.DataFrame(scaler_data1, index=index1, columns=['Energia_activa'])

index6 = np.arange(0, len(scaler_data6), 1)
scaler_df6 = pd.DataFrame(scaler_data6, index=index6, columns=['Energia_activa'])

index17 = np.arange(0, len(scaler_data17), 1)
scaler_df17 = pd.DataFrame(scaler_data17, index=index17, columns=['Energia_activa'])

In [ ]:
# Splitting the data for df_cliente1
train_size1 = int(len(scaler_df1) * 0.8)
train1, test1 = scaler_df1.iloc[0:train_size1], scaler_df1.iloc[train_size1:len(scaler_df1)]
print("Train size for df_cliente1:", len(train1), "Test size for df_cliente1:", len(test1))

# Splitting the data for df_cliente6
train_size6 = int(len(scaler_df6) * 0.8)
train6, test6 = scaler_df6.iloc[0:train_size6], scaler_df6.iloc[train_size6:len(scaler_df6)]
print("Train size for df_cliente6:", len(train6), "Test size for df_cliente6:", len(test6))

# Splitting the data for df_cliente17
train_size17 = int(len(scaler_df17) * 0.8)
train17, test17 = scaler_df17.iloc[0:train_size17], scaler_df17.iloc[train_size17:len(scaler_df17)]
print("Train size for df_cliente17:", len(train17), "Test size for df_cliente17:", len(test17))

In [ ]:
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)        
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

# Define the time steps
n_steps = 24

# Prepare training data
X_train1, y_train1 = create_dataset(train1, train1['Energia_activa'], n_steps)
X_train6, y_train6 = create_dataset(train6, train6['Energia_activa'], n_steps)
X_train17, y_train17 = create_dataset(train17, train17['Energia_activa'], n_steps)

# Prepare test data
X_test1, y_test1 = create_dataset(test1, test1['Energia_activa'], n_steps)
X_test6, y_test6 = create_dataset(test6, test6['Energia_activa'], n_steps)
X_test17, y_test17 = create_dataset(test17, test17['Energia_activa'], n_steps)

print("Training data shape:", X_train1.shape, y_train1.shape)
print("Test data shape:", X_test1.shape, y_test1.shape)
print(" ")
print("Training data shape:", X_train6.shape, y_train6.shape)
print("Test data shape:", X_test6.shape, y_test6.shape)
print(" ")
print("Training data shape:", X_train17.shape, y_train17.shape)
print("Test data shape:", X_test17.shape, y_test17.shape)

In [ ]:
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
# Model Architecture
rnn_model1 = Sequential([
    SimpleRNN(units=128,activation="tanh", input_shape=(X_train1.shape[1], X_train1.shape[2]), return_sequences=True),  # return_sequences=True if adding more RNN layers
    Dropout(0.2),  # Optional: Dropout for regularization
    SimpleRNN(units= 40,activation="tanh",return_sequences=True),
    Dropout(0.2),
    SimpleRNN(units= 40,activation="relu",return_sequences=False),
    Dropout(0.15),
    Dense(units=1)
])

# Compile the model
rnn_model1.compile(
    loss='mean_squared_error',
    optimizer=keras.optimizers.Adam(0.001)
)

# Callbacks
early_stopping1 = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint1 = ModelCheckpoint('best_rnn_model.keras', monitor='val_loss', save_best_only=True)

# Model Training
history = rnn_model1.fit(
    X_train1, y_train1,
    epochs=500,
    batch_size=24,
    validation_split=0.1,
    verbose=1,
    shuffle=False,
    callbacks=[early_stopping1, model_checkpoint1]
)


In [ ]:
from sklearn.metrics import r2_score
predictions = rnn_model1.predict(X_test1)
score = r2_score(y_test1,predictions)
print("R-Squared Score of RNN model",score)

In [ ]:
rnn_pred1 = rnn_model1.predict(X_test1)
df_rnn_final1 = test1[rnn_pred1.shape[0]*-1:].copy()
df_rnn_final1['Prediction'] = rnn_pred1[:,0]
df_reset_index1 = df_cliente1.reset_index()
df_rnn_final1['Fecha_hora'] = df_reset_index1['Fecha_hora'][rnn_pred1.shape[0]*-1:]
df_rnn_final1 = df_rnn_final1.set_index(["Fecha_hora"], drop=True)

In [ ]:
df_rnn_final1['Energia_activa'] = scaler1.inverse_transform(df_rnn_final1['Energia_activa'].to_numpy().reshape(-1, 1))
df_rnn_final1['Prediction'] = scaler1.inverse_transform(df_rnn_final1['Prediction'].to_numpy().reshape(-1, 1))

In [ ]:
train1_2 = train1.copy()
train1_2.index = df_cliente1.index[:len(train1)]
train1_2['Energia_activa'] = scaler1.inverse_transform(train1_2['Energia_activa'].to_numpy().reshape(-1, 1))
train1_2['Fecha_hora'] = df_cliente1['Fecha_hora']
train1_2 = train1_2.set_index(['Fecha_hora'])

In [ ]:
def report_metrics(y_true, y_pred):
    explained_variance = metrics.explained_variance_score(y_true, y_pred)
    mae = metrics.mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(metrics.mean_squared_error(y_true, y_pred))
    mse = metrics.mean_squared_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    print("Explained Variance:", explained_variance)
    print("MAE:", mae)
    print("RMSE:", rmse)
    print("MSE:", mse)
    print("MAPE:", mape)

In [ ]:
plt.figure(figsize=(50,15))
plt.title('TensorFlow - Keras RNN Model')
plt.plot(train1_2, color='red', label='Actual Energia_activa (Train)')
plt.xlabel('Fecha_hora')
plt.ylabel('Energia_activa')
plt.plot(df_rnn_final1['Energia_activa'], color='green', label='Actual Energia_activa (Test)')
plt.plot(df_rnn_final1['Prediction'], color='purple', label='Predicted Energia_activa (LSTM)')
plt.legend()

plt.show()

report_metrics(df_rnn_final1['Energia_activa'].squeeze(), df_rnn_final1['Prediction'].squeeze())

In [ ]:
from keras.models import load_model
# Guardar el modelo
rnn_model1.save('rnn_model1.h5')

## Cliente 6

In [ ]:
# Model Architecture
rnn_model6 = Sequential([
    SimpleRNN(units=128, input_shape=(X_train6.shape[1], X_train6.shape[2])),  # return_sequences=True if adding more RNN layers
    Dropout(0.1),  # Optional: Dropout for regularization
    Dense(units=1)
])

In [ ]:
# Compile the model
rnn_model6.compile(
    loss='mean_squared_error',
    optimizer=keras.optimizers.Adam(0.001)
)

In [ ]:
# Callbacks
early_stopping6 = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint6 = ModelCheckpoint('best_rnn_model.keras', monitor='val_loss', save_best_only=True)

In [ ]:
# Model Training
history = rnn_model6.fit(
    X_train6, y_train6,
    epochs=200,
    batch_size=1000,
    validation_split=0.1,
    verbose=1
)

In [ ]:
from sklearn.metrics import r2_score
predictions = rnn_model6.predict(X_test6)
score = r2_score(y_test6,predictions)
print("R-Squared Score of RNN model",score)

In [ ]:
rnn_pred6 = rnn_model6.predict(X_test6)
df_rnn_final6 = test6[rnn_pred6.shape[0]*-1:].copy()
df_rnn_final6['Prediction'] = rnn_pred6[:,0]
df_reset_index6 = df_cliente6.reset_index()
df_rnn_final6['Fecha_hora'] = df_reset_index6['Fecha_hora'][rnn_pred6.shape[0]*-1:]
df_rnn_final6 = df_rnn_final6.set_index(["Fecha_hora"], drop=True)

In [ ]:
df_rnn_final6['Energia_activa'] = scaler6.inverse_transform(df_rnn_final6['Energia_activa'].to_numpy().reshape(-1, 1))
df_rnn_final6['Prediction'] = scaler6.inverse_transform(df_rnn_final6['Prediction'].to_numpy().reshape(-1, 1))

In [ ]:
train6_2 = train6.copy()
train6_2.index = df_cliente6.index[:len(train6)]
train6_2['Energia_activa'] = scaler6.inverse_transform(train6_2['Energia_activa'].to_numpy().reshape(-1, 1))
train6_2['Fecha_hora'] = df_cliente6['Fecha_hora']
train6_2 = train6_2.set_index(['Fecha_hora'])

In [ ]:
plt.figure(figsize=(50,15))
plt.title('TensorFlow - Keras RNN Model')
plt.plot(train6_2, color='red', label='Actual Energia_activa (Train)')
plt.xlabel('Fecha_hora')
plt.ylabel('Energia_activa')
plt.plot(df_rnn_final6['Energia_activa'], color='green', label='Actual Energia_activa (Test)')
plt.plot(df_rnn_final6['Prediction'], color='purple', label='Predicted Energia_activa (LSTM)')
plt.legend()

plt.show()

report_metrics(df_rnn_final6['Energia_activa'].squeeze(), df_rnn_final6['Prediction'].squeeze())

In [ ]:
plt.figure(figsize=(50,15))
plt.title('TensorFlow - Keras RNN Model')
plt.plot(df_rnn_final6['Energia_activa'], color='green', label='Actual Energia_activa (Test)')
plt.plot(df_rnn_final6['Prediction'], color='purple', label='Predicted Energia_activa (LSTM)')
plt.xlabel('Fecha_hora')
plt.ylabel('Energia_activa')
plt.legend()
plt.show()

report_metrics(df_rnn_final6['Energia_activa'].squeeze(), df_rnn_final6['Prediction'].squeeze())

## Cliente 17

In [ ]:
# Model Architecture
rnn_model17 = Sequential([
    SimpleRNN(units=128,activation="tanh", input_shape=(X_train17.shape[1], X_train17.shape[2]), return_sequences=True),  # return_sequences=True if adding more RNN layers
    Dropout(0.2),  # Optional: Dropout for regularization
    SimpleRNN(units= 128,activation="tanh",return_sequences=True),
    Dropout(0.2),
    SimpleRNN(units= 64,activation="tanh",return_sequences=False),
    Dropout(0.2),
    Dense(units=1)
])

In [ ]:
# Compile the model
rnn_model17.compile(
    loss='mean_squared_error',
    optimizer=keras.optimizers.Adam(0.001)
)

In [ ]:
# Callbacks
early_stopping17 = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint17 = ModelCheckpoint('best_rnn_model.keras', monitor='val_loss', save_best_only=True)

In [ ]:
# Model Training
history = rnn_model17.fit(
    X_train17, y_train17,
    epochs=500,
    batch_size=24,
    validation_split=0.1,
    verbose=1,
    shuffle=False,
    callbacks=[early_stopping17, model_checkpoint17]
)

In [ ]:
from sklearn.metrics import r2_score
predictions = rnn_model17.predict(X_test17)
score = r2_score(y_test17,predictions)
print("R-Squared Score of RNN model",score)

In [ ]:
rnn_pred17 = rnn_model17.predict(X_test17)
df_rnn_final17 = test17[rnn_pred17.shape[0]*-1:].copy()
df_rnn_final17['Prediction'] = rnn_pred17[:,0]
df_reset_index17 = df_cliente17.reset_index()
df_rnn_final17['Fecha_hora'] = df_reset_index17['Fecha_hora'][rnn_pred17.shape[0]*-1:]
df_rnn_final17 = df_rnn_final17.set_index(["Fecha_hora"], drop=True)

In [ ]:
df_rnn_final17['Energia_activa'] = scaler17.inverse_transform(df_rnn_final17['Energia_activa'].to_numpy().reshape(-1, 1))
df_rnn_final17['Prediction'] = scaler17.inverse_transform(df_rnn_final17['Prediction'].to_numpy().reshape(-1, 1))

In [ ]:
train17_2 = train17.copy()
train17_2.index = df_cliente17.index[:len(train17)]
train17_2['Energia_activa'] = scaler17.inverse_transform(train17_2['Energia_activa'].to_numpy().reshape(-1, 1))
train17_2['Fecha_hora'] = df_cliente17['Fecha_hora']
train17_2 = train17_2.set_index(['Fecha_hora'])

In [ ]:
plt.figure(figsize=(50,15))
plt.title('TensorFlow - Keras RNN Model')
plt.plot(train17_2, color='red', label='Actual Energia_activa (Train)')
plt.xlabel('Fecha_hora')
plt.ylabel('Energia_activa')
plt.plot(df_rnn_final17['Energia_activa'], color='green', label='Actual Energia_activa (Test)')
plt.plot(df_rnn_final17['Prediction'], color='purple', label='Predicted Energia_activa (LSTM)')
plt.legend()

plt.show()

report_metrics(df_rnn_final17['Energia_activa'].squeeze(), df_rnn_final17['Prediction'].squeeze())

In [ ]:
plt.figure(figsize=(50,15))
plt.title('TensorFlow - Keras RNN Model')
plt.plot(df_rnn_final17['Energia_activa'], color='green', label='Actual Energia_activa (Test)')
plt.plot(df_rnn_final17['Prediction'], color='purple', label='Predicted Energia_activa (LSTM)')
plt.xlabel('Fecha_hora')
plt.ylabel('Energia_activa')
plt.legend()
plt.show()

report_metrics(df_rnn_final17['Energia_activa'].squeeze(), df_rnn_final17['Prediction'].squeeze())

In [ ]:
plt.figure(figsize=(50,15))
plt.title('TensorFlow - Keras RNN Model')
plt.plot(df_rnn_final17['Energia_activa'].loc['2023-02-01':'2023-02-03'], color='green', label='Actual Energia_activa (Test)')
plt.plot(df_rnn_final17['Prediction'].loc['2023-02-01':'2023-02-03'], color='purple', label='Predicted Energia_activa (LSTM)')
plt.xlabel('Fecha_hora')
plt.ylabel('Energia_activa')
plt.legend()
plt.show()

In [ ]:
from keras.models import load_model
# Guardar el modelo
rnn_model6.save('rnn_model6.h5')
rnn_model17.save('rnn_model17.h5')